In [1]:
import numpy as np
import json
from datetime import *
from PIL import Image
import os
import os.path as osp
import zipfile as zf
import io
import multiprocessing

DATA_DIR = osp.abspath('../../data/gta-data/dataset')
PCL_DIR = osp.abspath('../../data/gta-data/pcl')
BBOX = (50, 50, 50)
CAMERAS = np.array([[0, -0.4, 2, 1], [0 , -0.8, 0.6, 1], [0, -0.6, 2, 1], [0, -0.8, 0.6, 1]])
DATA_SIZE = -1
THREADS = 4

In [2]:
ALL_FILES = [osp.join(DATA_DIR, f) for f in os.listdir(DATA_DIR)]
JSONS = sorted([f for f in ALL_FILES if f.endswith('.json')])
DEPTHS = sorted([f for f in ALL_FILES if '-depth' in f])

assert len(DEPTHS) == len(JSONS)

if DATA_SIZE is not -1:
    DEPTHS = DEPTHS[:DATA_SIZE]
    JSONS = JSONS[:DATA_SIZE]

89196


In [ ]:
def depth_to_pcl(depth, proj_matrix, view_matrix, bbox):
    height, width = depth.shape
    y, x = np.where(depth > 0.0)
    y_data = (-2 / height) * y + 1
    x_data = (2 / width) * x - 1
    points = np.ones((4, len(y)))
    points[0,:] = y_data
    points[1,:] = x_data
    points[2,:] = depth[y, x]
    
    ego_points = np.linalg.inv(proj_matrix) @ points
    ego_points /= ego_points[3,:]
    
    ego_points = ego_points[:,(np.abs(ego_points[0,:]) < bbox[0]) & (np.abs(ego_points[1,:]) < bbox[1]) & (np.abs(ego_points[2,:]) < bbox[2])]
    
    world_points = np.linalg.inv(view_matrix) @ ego_points
    world_points /= world_points[3,:]
    
    return world_points[:3,:].astype(np.float32)

In [ ]:
def process(i, depth_im, json_file):
    global BBOX, CAMERAS, PCL_DIR
    with open(json_file, mode='r') as f:
        json_data = json.load(f)
    del json_data['entities']
    del json_data['height']
    del json_data['width']
    del json_data['timestamp']
    del json_data['camera_fov']
    del json_data['snapshot_id']
    depth_int = np.array(Image.open(depth_im), dtype=np.uint16)
    depth_real = np.array(depth_int/np.iinfo(np.uint16).max, dtype=np.float32)
    proj_matrix = np.array(json_data['proj_matrix'])
    view_matrix = np.array(json_data['view_matrix'])
    world = depth_to_pcl(depth_real, proj_matrix, view_matrix, BBOX)
    car_center = ((np.linalg.inv(view_matrix) @ CAMERAS[i % 4,:]).T)[:3].tolist()
    tmp_meta = {'timestamp': datetime.strptime(json_data['imagepath'], '%Y-%m-%d--%H-%M-%S--%f').timestamp(),
                    'pcl_id' : i,
                    'bbox' : BBOX,
                    'car_center' : car_center,
                    'orig_json' : json_data}
    with zf.ZipFile(osp.join(PCL_DIR,'%05d.zip' % (i,)), 'w', zf.ZIP_LZMA) as cf:
        cf.writestr('metadata.json', json.dumps(tmp_meta))
        tmpf = io.BytesIO()
        np.save(tmpf, world)
        cf.writestr('pcl.npy', tmpf.getvalue())

In [3]:
os.makedirs(PCL_DIR, exist_ok=True)
with multiprocessing.Pool(THREADS) as pool:
    pool.starmap(process, zip(range(len(DEPTHS)), DEPTHS, JSONS))